# 繪製2019年台灣出生率地圖
2021/03/05 蘇彥庭

## 一、前言

最近有在計畫要參加台北市電腦公會主辦的[資料創新應用競賽](https://opendata-contest.tca.org.tw/)，這個競賽是透過政府開放資料等各式數據，來協助解決施政議題及提升公部門服務品質。

近年來少子女化的議題愈來愈嚴重，最近的新聞「[台灣史上最低! 元月新生兒數不足萬人](https://news.cts.com.tw/cts/general/202103/202103012032961.html)」，對於台灣來說是一個嚴重的國安危機。

我們這組針對這次競賽，目前有在考慮進行少子女化相關的議題研究與應用。這是我先用Python試做的一個互動式地圖，主要依據政府開放資料提供的[鄉鎮市區界線地圖資料](https://data.gov.tw/dataset/7441)和[社會經濟資料服務平台](https://segis.moi.gov.tw/STAT/Web/Platform/QueryInterface/STAT_QueryTopProduct.aspx)，來繪製一個2019年台灣的出生率地圖。

* 資料來源：
* [政府公開資料觀測站-鄉鎮市區界線(TWD97經緯度)](https://data.gov.tw/dataset/7441)
* [社會經濟資料服務平台-常用資料下載-行政區](https://segis.moi.gov.tw/STAT/Web/Platform/QueryInterface/STAT_QueryTopProduct.aspx)

## 二、整理鄉鎮市區圖資檔案

在繪製動態地圖時，我主要參考Python Plotly套件官網的[Mapbox Choropleth Maps in Python](https://plotly.com/python/mapbox-county-choropleth/)教學頁面，依據裡面的做法來進行繪製。

透過Plotly套件的`choropleth_mapbox`函數繪製地圖時，需要有GeoJSON檔案。

> GeoJSON是一種基於JSON的地理空間數據交換格式，它定義了幾種類型JSON對象以及它們組合在一起的方法，以表示有關地理要素、屬性和它們的空間範圍的數據。
> (文字來源: [維基百科](https://zh.wikipedia.org/wiki/GeoJSON))

在政府開放資料網站中，有提供台灣的[鄉鎮市區界線地圖資料](https://data.gov.tw/dataset/7441)，但這個資料的格式是SHP格式。

> ESRI Shapefile（shp），或簡稱shapefile，是美國環境系統研究所公司（ESRI）開發的空間資料開放格式。[1]目前，該檔案格式已經成為了地理訊息軟體界的開放標準，這表明ESRI公司在全球的地理訊息系統市場的重要性。Shapefile也是重要的交換格式，能夠在ESRI與其他公司的產品之間進行資料互操作。
> (文字來源: [維基百科](https://zh.wikipedia.org/wiki/Shapefile))

所以我們必須要先將政府提供的SHP格式地圖資料，轉為GeoJSON格式。經過一番搜索與測試，發現Python的[geopandas套件](https://geopandas.org/)可以解決這個轉換的問題。

首先安裝geopandas套件時，會遇到這個問題：

[Error installing geopandas:“ A GDAL API version must be specified ” in Anaconda](https://stackoverflow.com/questions/54734667/error-installing-geopandas-a-gdal-api-version-must-be-specified-in-anaconda)

按照問題的解法，安裝以下套件清單即可順利安裝geopandas套件：

```
pip install wheel
pip install pipwin
pipwin install numpy
pipwin install pandas
pipwin install shapely
pipwin install gdal
pipwin install fiona
pipwin install pyproj
pipwin install six
pipwin install rtree
pipwin install geopandas
```

安裝好後，載入本篇文章所需的套件清單：

In [ ]:
# 載入套件
import geopandas
import json
import numpy as np
import pandas as pd
import plotly.express as px

接下來將下載後的[鄉鎮市區界線地圖資料](https://data.gov.tw/dataset/7441)解壓縮後，用Python讀取`
TOWN_MOI_1091016.shp`檔案，並轉換為geojson格式：

In [ ]:
# 將shp檔案格式轉為geoJson檔案格式
shp_file = geopandas.read_file('TOWN_MOI_1091016.shp', encoding='utf8')
shp_file.to_file('TOWN_MOI_1091016.geojson', driver='GeoJSON', epsg=4326)  # 經緯度由epsg3826(TWD97)轉換為epsg4326(WGS84)

利用json模組，讀入geojson檔案：

In [ ]:
# 讀取geojson
with open('TOWN_MOI_1091016.geojson', encoding='utf8') as response:
    mapGeo = json.load(response)

In [ ]:
# 觀察geojson資料狀況
print(mapGeo['features'][0]['properties'])
print(mapGeo['features'][1]['properties'])

此處依據geojson檔案內記載的鄉鎮市區清單資訊，建立一個Pandas表格，待會要將此表做為主表，把出生率資料併進來：

In [ ]:
# 依據圖資資料建立鄉鎮市區清單
df = pd.DataFrame.from_dict([i['properties'] for i in mapGeo['features']])
df.head()

# 三、整理出生率資料

SEGIS(社會經濟資料服務平台)提供的「108年內政大數據資料應用組競賽用資料集_村里.csv」檔案，裡面函蓋到台灣每個村里的資訊。

由於此處是要繪製鄉鎮市區的出生率資料，所以需要透過`group_by()`來彙整村里的出生人數與人口數資訊，並計算出各鄉鎮市區的出生率。

In [ ]:
# 讀取108年內政大數據資料應用組競賽用資料集_村里.csv
segisData = pd.read_csv('108年內政大數據資料應用組競賽用資料集_村里.csv', 
                        skiprows=1, 
                        dtype={'鄉鎮市區代碼': 'object', '村里代碼': 'object'})
segisData.head()

我在做併表繪製地圖後，發現金門縣的資料沒有被繪製出來。

經過檢查發現，原來是內政資料的金門縣鄉鎮市區代碼前面少一個0，導致和圖資資料的鄉鎮市區代碼不一致，沒有被併到。

所以此處我做一個判斷，如果鄉鎮市區代碼只有7個位數，則在前面補0，避免待會併表失敗。

In [ ]:
# 為能夠對應其他來源資料 鄉鎮市區代碼若只有7碼 則第一碼補0
# 此為108年內政大數據資料應用組競賽用資料集_村里資料問題
reviseIdx = segisData['鄉鎮市區代碼'].str.len() == 7
segisData.loc[reviseIdx, '鄉鎮市區代碼'] = '0' + segisData.loc[reviseIdx, '鄉鎮市區代碼']

利用`group_by()`計算鄉鎮市區出生率資訊：

In [ ]:
# 選擇計算出生率指標所需的資料
obsData = segisData[['鄉鎮市區代碼', '人口數', '出生數']]
# 因資料為村里統計資訊 所以利用group_by將村里資訊彙整成鄉鎮市區資訊
obsData = obsData.groupby(['鄉鎮市區代碼']).agg({'人口數': 'sum', '出生數': 'sum'}).reset_index()
# 計算鄉鎮市區出生率資訊
obsData['出生率(‰)'] = round(obsData['出生數']/obsData['人口數']*1000, 2)
# 將鄉鎮市區代碼轉為文字
obsData['鄉鎮市區代碼'] = obsData['鄉鎮市區代碼'].astype(str)
obsData.head()

## 四、圖資資料與出生率資料併表

以圖資資料為主表，left join併入出生率資料，key為鄉鎮市區代碼。

併表完後，此處額外將縣市名稱欄位和鄉鎮市區名稱欄位合併成一個新欄位，待會繪製地圖時方便使用者來辨識。

In [ ]:
# 合併資料
df = df.merge(obsData, how='left', left_on='TOWNCODE', right_on='鄉鎮市區代碼')
# 產生完整鄉鎮市區名稱資訊
df['鄉鎮市區名稱'] = df['COUNTYNAME'] + df['TOWNNAME']
df.head()

# 五、繪製出生率地圖

整理好資料後，即可繪製台灣2019年鄉鎮市區出生率地圖：

In [ ]:
# 繪製出生率地圖
fig = px.choropleth_mapbox(df,                                                # 資料表
                           geojson=mapGeo,                                    # 地圖資訊
                           locations='TOWNCODE',                              # df要對應geojson的id名稱
                           featureidkey='properties.TOWNCODE',                # geojson對應df的id名稱
                           color='出生率(‰)',                                 # 顏色區分對象
                           color_continuous_scale='Viridis',                  # 設定呈現的顏色
                           range_color=(round(np.nanmin(df['出生率(‰)'])),    # 顏色的值域範圍
                                        round(np.nanmax(df['出生率(‰)']))),   
                           mapbox_style='carto-positron',                    # mapbox地圖格式
                           zoom=6,                                           # 地圖縮放大小: 數字愈大放大程度愈大
                           center={'lat': 23.5832, 'lon': 120.5825},         # 地圖中心位置: 此處設定台灣地理中心碑經緯度
                           opacity=0.5,                                               # 設定顏色區塊的透明度 數值愈大愈不透明
                           hover_data=['鄉鎮市區名稱', '出生率(‰)', '人口數', '出生數']  # 設定游標指向資訊
                          )
fig.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
fig.show()